In [ ]:
# Imports
%%capture

!pip install --upgrade tensorflow
!pip install tensorflow-gpu
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datasets
!pip install tokenizers
!pip install git+https://github.com/huggingface/transformers
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu
!pip install wget

# !python -m wget https://github.com/huggingface/transformers/tree/master/src/transformers/trainer_seq2seq.py
# !python -m wget https://github.com/huggingface/transformers/tree/master/src/transformers/training_args_seq2seq.py

In [ ]:
# Imports
import pandas as pd
import datasets
import csv
import transformers as ft

from IPython.display import display, HTML
from transformers import EncoderDecoderModel, BertTokenizer, BertTokenizerFast
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import ClassLabel

In [ ]:
# Test CUDA
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print("Device:", device)

Device: cuda


In [ ]:
# Load swiss data
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)
path_corpus = path_drive + "/My Drive/Temp/Corpus/data_train.csv"

data_txt = []
data_ref = []

with open(path_corpus, "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_ALL)
    next(reader, None)

    for row in reader:
        data_txt.append(row[0])
        data_ref.append(row[1])

tuples = list(zip(data_txt, data_ref))
dataframe = pd.DataFrame(tuples, columns=["article", "highlights"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load german data
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)
path_corpus = path_drive + "/My Drive/Temp/Corpus/data_train_test.xlsx"

tuples = pd.read_excel(path_corpus)
del tuples["Unnamed: 0"]
dataframe = pd.DataFrame(tuples, columns=["article", "highlights"]) # dataframe = pd.concat([dataframe, tuples])
dataframe = dataframe.dropna()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Clean redactional data
print(len(dataframe))
dataframe = dataframe[~dataframe["highlights"].str.contains("ZEIT")]
print(len(dataframe))

12408
12408


In [ ]:
# Concat swiss and german data
german_data = datasets.arrow_dataset.Dataset.from_pandas(dataframe[["article", "highlights"]])
german_data = german_data.shuffle()
print(dataframe)

                                                 article                                         highlights
0      Israelische Kampfjets haben in der Nacht Ziele...   Als Reaktion auf Attacken aus dem Gazastreife...
1      Monty Ott ist Vorsitzender von Keshet Deutschl...   Für Jüdinnen und Juden hat Deutschland nur di...
2      Was können Sie tun?\n- Sie können zur Startsei...  In seiner Amtszeit habe er eine “Mentalität de...
3      Es gibt das Gerücht, dass Xi Jinping in Wuhan ...   China wird von der Kommunistischen Partei aut...
4      Wie konnten sie nur, die Anführenden der Demok...   Die Wahlnacht von Iowa hat alle Beteiligten b...
...                                                  ...                                                ...
12533  Das in der Kritik stehende Kommando Spezialkrä...  KSK-Soldaten konnten laut Medienrecherchen unt...
12534  Franka Lu ist eine chinesische Journalistin un...  China hat die Pandemie erfolgreich unterdrückt...
12535  Nach drei Angriffen m

In [ ]:
# Split data
train_size = int(len(dataframe) * 0.9)
valid_size = int(len(dataframe) * 0.075)
test_size = int(len(dataframe) * 0.025)

train_data = german_data.select(range(0, train_size))
val_data = german_data.select(range(train_size, train_size + valid_size))
test_data = german_data.select(range(train_size + valid_size, len(dataframe)))

In [ ]:
# Load english data
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:5%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:2%]")

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0a01b1abede4f646130574f203de57a293ded8a7a11e3406a539453afdfeb2c0)
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0a01b1abede4f646130574f203de57a293ded8a7a11e3406a539453afdfeb2c0)
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0a01b1abede4f646130574f203de57a293ded8a7a11e3406a539453afdfeb2c0)


In [ ]:
# Explore corpus
df = pd.DataFrame(train_data)

text_list = []
summary_list = []

for index, row in df.iterrows():
    text = row["article"]
    summary = row["highlights"]
    text_list.append(len(text))
    summary_list.append(len(summary))
    
print(sum(text_list) / len(text_list))
print(sum(summary_list) / len(summary_list))

3915.6129
264.5824


In [ ]:
# Explore corpus
train_data.info.description
df = pd.DataFrame(train_data[:1])

# del df["id"]

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html()))

In [ ]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

In [ ]:
# Prepare data
encoder_max_length = 512
decoder_max_length = 128
batch_size = 4 # 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [ ]:
# Training data
train_data = train_data.shuffle()

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights"] # "id"
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0a01b1abede4f646130574f203de57a293ded8a7a11e3406a539453afdfeb2c0/cache-7af9f5bf341e5915.arrow


In [ ]:
# Validation data
val_data = val_data.shuffle()

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights"] # id
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

val_data = val_data.select(range(2000))

In [ ]:
# Load models
tf2tf = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased", tie_encoder_decoder=False)
tf2tf.save_pretrained("bert2bert_multilingual")
tf2tf = EncoderDecoderModel.from_pretrained("bert2bert_multilingual")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [ ]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size # tokenizer.vocab_size

In [ ]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [ ]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# Load checkpoint
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)

path_output = path_drive + "/My Drive/Temp/Models"
path_checkpoint = path_output + "/checkpoint-4000"

tf2tf = EncoderDecoderModel.from_pretrained(path_checkpoint)
tf2tf.to("cuda")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# Empty cache
import psutil

torch.cuda.empty_cache()
psutil.virtual_memory()

svmem(total=13653569536, available=8010371072, percent=41.3, used=8964763648, free=168841216, active=9098362880, inactive=3929788416, buffers=35004416, cached=4484960256, shared=13643776, slab=187285504)

In [ ]:
# Setup arguments
training_args = ft.Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir=path_output, # "./"
    warmup_steps=1000,
    save_steps=2000,
    logging_steps=100,
    eval_steps=2000,
    save_total_limit=1,
    fp16=True
)

In [ ]:
# Start training
trainer = ft.Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=750, training_loss=5.587833760579427, metrics={'train_runtime': 585.0085, 'train_samples_per_second': 1.282, 'total_flos': 4425924222720000.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 4096, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 15745024, 'train_mem_gpu_alloc_delta': 4702387712, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 4992374272})

In [ ]:
# Load checkpoint
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)

path_output = path_drive + "/My Drive/Temp/Models"
path_checkpoint = path_output + "/checkpoint-4000"

tf2tf = EncoderDecoderModel.from_pretrained(path_checkpoint)
tf2tf.to("cuda")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# Evaluate training
def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred_summary"] = output_str

    return batch

results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article"]
)

print(results[0]["pred_summary"])
print(results[0]["highlights"])
print("====================")

rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid


NEW : " It's the same same - sex sex sex, " police says. The boy's father's mother says she's a teenager. The girl's name's " I'm not a girl, " says says. She's not a teen teen's girlfriend, " she says.
James Best, who played the sheriff on "The Dukes of Hazzard," died Monday at 88 .
"Hazzard" ran from 1979 to 1985 and was among the most popular shows on TV .


Score(precision=0.009788744117622783, recall=0.013462376021361085, fmeasure=0.011080057178249322)